# "Настройка" BEIR

In [2]:
!pip install datasets
#!pip install beir
#!pip install gdown

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-16.1.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.4.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 1.5 MB/s eta 0:00:0000:0100:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.7 M

In [ ]:
!git clone git@github.com:kngrg/rus_beir.git

In [15]:
import os
import gdown
import zipfile

#full corpus
#url = "https://drive.google.com/uc?export=download&id=1RkR7YQ2AewNg96MmHF7sksCC_oxxob5V"

#corpus_100
url = 'https://drive.google.com/uc?export=download&id=1l-dVU6GSKz5REXbOZ3JR4MEKV0TO0VKJ'

out_dir = os.path.join(os.getcwd(), "datasets")
os.makedirs(out_dir, exist_ok=True)
#output_file_path = os.path.join(out_dir, "rus-mmarco.zip")
output_file_path = os.path.join(out_dir, "rus-mmarco-mini.zip")

gdown.download(url, output_file_path, quiet=False)

#zip_path = os.path.join(out_dir, "rus-mmarco.zip")
zip_path = os.path.join(out_dir, "rus-mmarco-mini.zip")

print(zip_path)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(out_dir)
print(f"Dataset extracted here: {out_dir}")

Downloading...
From: https://drive.google.com/uc?export=download&id=1l-dVU6GSKz5REXbOZ3JR4MEKV0TO0VKJ
To: /kaggle/working/datasets/rus-mmarco-mini.zip
100%|██████████| 26.1k/26.1k [00:00<00:00, 22.5MB/s]

/kaggle/working/datasets/rus-mmarco-mini.zip
Dataset extracted here: /kaggle/working/datasets


In [15]:
from tqdm import tqdm
def CorpusToDict(corpus):
    corpus_dict = {}
    for row in tqdm(corpus, desc="corpus->dict"):
        corpus_dict[str(row['id'])] = {'text': row['text'], 'title': row['title']}
    return corpus_dict

def QueriesToDict(queries):
    queries_dict = {}
    for row in tqdm(queries, desc="queries->dict"):
        queries_dict[str(row['id'])] = row['text']
    return queries_dict

In [17]:
# Загрузка через HF
from beir.datasets.data_loader_hf import HFDataLoader
corpus, queries, qrels = HFDataLoader(hf_repo="kngrg/rus-mmarco-mini", hf_repo_qrels="kngrg/rus-mmarco-mini-qrels", streaming=False,
                                                  keep_in_memory=False).load(split='train')
corpus = CorpusToDict(corpus)
queries = QueriesToDict(queries)

Generating train split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/8841823 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/909824 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/532751 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/7437 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/532751 [00:00<?, ? examples/s]

Map:   0%|          | 0/532751 [00:00<?, ? examples/s]

Filter:   0%|          | 0/909824 [00:00<?, ? examples/s]

queries->dict: 100%|██████████| 502939/502939 [00:54<00:00, 9240.46it/s]


In [16]:
from beir.datasets.data_loader import GenericDataLoader

data_path = "datasets/rus-mmarco-mini"
corpus1, queries1, qrels1 = GenericDataLoader(data_path).load(split="train") # or split = "train"

  0%|          | 0/100 [00:00<?, ?it/s]

# ElasticSearch

In [32]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [33]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [34]:
import time

# Sleep for few seconds to let the instance start.
time.sleep(20)

In [35]:
%%bash

ps -ef | grep elasticsearch

root        1840    1838  0 17:01 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon      1841    1840 99 17:01 ?        00:00:34 /kaggle/working/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-6607876387477467146 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:

In [36]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "df5a4fb3d0b2",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "a0GcwbHvTq-vRQ3iQopnzQ",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

#### Provide parameters for elastic-search
hostname = "localhost"
index_name = "mmarco"
initialize = True # True, will delete existing index with same name and reindex all documents

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize, language="russian")
retriever = EvaluateRetrieval(model)

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)



  0%|          | 0/8841823 [00:00<?, ?docs/s]

  0%|          | 1/8841823 [00:21<53198:06:07, 21.66s/docs]

  0%|          | 501/8841823 [00:21<76:01:27, 32.30docs/s] 

  0%|          | 1001/8841823 [00:22<32:13:49, 76.19docs/s]

  0%|          | 1501/8841823 [00:22<18:04:42, 135.83docs/s]

  0%|          | 2001/8841823 [00:22<11:22:28, 215.88docs/s]

  0%|          | 2501/8841823 [00:22<7:38:31, 321.29docs/s] 

  0%|          | 3001/8841823 [00:23<5:22:04, 457.39docs/s]

  0%|          | 3501/8841823 [00:23<3:56:05, 623.92docs/s]

  0%|          | 4001/8841823 [00:23<3:02:48, 805.73docs/s]

  0%|          | 4501/8841823 [00:23<2:23:50, 1023.95docs/s]

  0%|          | 5001/8841823 [00:24<1:57:13, 1256.42docs/s]

  0%|          | 5501/8841823 [00:24<1:37:44, 1506.63docs/s]

  0%|          | 6001/8841823 [00:24<1:23:14, 1769.14docs/s]

  0%|          | 6501/8841823 [00:24<1:13:16, 2009.76docs/s]

  0%|          | 7001/8841823 [00:24<1:08:33, 2147.90docs/s]

  0%|          | 7501/88418

In [21]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

In [ ]:
metrics = {"ndcg": ndcg, "_map": _map, "recall": recall, "precision": precision}
for metric in metrics.keys():
    for it_num, it_val in zip(metrics[metric], metrics[metric].values()):
        print(it_num, it_val )
    print('\n')
